$\textbf{(a):}$

Part A of the algorithm gives the $LDL^{t}$ factorization of matrix $A$ and is taken from Burden et al (page 422).
Part B gives the solution (and the modification required) to the linear system by performing forward/backward substituitions and is taken from Burden et al (page 425). 

Input: matrix A, vector b

Output: matrix L (lower-triangular), matrix D (diagonal), vector x (solution)

$\textbf{Part A}$:

$\textbf{START}$

(Step 1): For $i = 1,...,n$ do steps 2 - 4:

(Step 2): For $j = 1,...,i-1$  set : $v_j = l_{ij}d_j$

(Step 3): Set $d_i = a_{ii} - \sum_{j=1}^{i-1}l_{ij}v_j$

(Step 4): For $j=i+1,...,n$ set : $l_{ji} = (a_{ji} - \sum_{k=1}^{i-1}l_{jk}v_k) / d_i $

(Step 5): Return : $l_ij$ for $j= 1,...,i-1$ and $i=1,...,n$ ; $d_i$ for $i=1,...,n$


$\textbf{Part B}$ :

(Step 6): Set : $y_1 = b_1$

(Step 7): For $i=2,...,n$ set : $y_i = b_i - \sum_{j=1}^{i-1}l_{ij}y_j$ 

(Step 8): For $i=1,...,n$ set : $z_i = y_i / d_i$

(Step 9): Set : $x_n = z_n$

(Step 10): For $i =n-1,...,1$ set : $x_i = z_i - \sum_{j=i+1}^{n}l_{ji}x_j$

(Step 11): Return : $x_i$ for $i=1,...,n$

$\textbf{END}$

Part B essentially computes the solutions to the following systems (in order):

1) Find $y$ from $Ly=b$ (steps: 6 & 7)

2) Find $z$ from $Dz=y$ (step: 8)

3) Find $x$ from $L^{t} x=z$ (steps: 9 & 10) $\longrightarrow $  vector $x$ is the required solution!

$\textbf{(b):}$

This method was implemented (from scratch and based on the pseudocode in part (a)) in programming language R. Using function definition (defined below as: 'ldlt_func(A, b)') we make the method repeatable for different linear systems.

In [2]:
# Implementing the method as a function, 'ldlt_func()' :
  # Note: operands '<-' (i.e. the assignment operator) and '=' are equivalent in R

ldlt_func <- function(A, b){
    
  # Extracting the dimesnions of matrix A:
  n <- dim(A)[1]

  # Containers for intermediate values:
  # 1 x n matrices
  v <- matrix(c(0), nrow = 1, ncol = n) 
  d <- matrix(c(0), nrow = 1, ncol = n)
  # n x n matrix
  l2 <- matrix(c(0), nrow = n, ncol = n)     
  x <- matrix(c(0), nrow = n, ncol = 1)
  y <- matrix(c(0), nrow = n, ncol = 1)
  z <- matrix(c(0), nrow = n, ncol = 1)
  l <- A
  b <- b
  
  # PART A (LDLt Factorization of Matrix A):
  
  # Step 1:
  for (i in 1:n){
    
    # Step 2:
    for (j in 1:i){
      v[j] = (l[i, j]*d[j])
    }
      
    # Step 3:
    d[i] = l[i,i]
    for (j in 1:i){
      d[i] = d[i] - sum(l[i,j]*v[j])  
    }
      
    # Step 4:
    for (j in i:n){
      for (k in 1:i){
        l[j,i] = l[j,i] - sum(l[j,k]*v[k])  
      }
      l[j,i] = l[j,i]/d[i]   
    }
  }
    
  # Step 5:
  # The correct form (for presentation) of the lower-triangular matrix L :
  for (i in 1:n){
    for (j in 1:i){
      l2[i, j] = l[i,j]
    }
  }
  
  # The correct form of the diagonal matrix D :
  d <- diag(d[,])
  
  # Optional: check whether the factorization is correct (i.e. if LDLt==A):
  # print(A)
  # print((l2%*%d)%*%(t(l2)))
  
    
  # PART B (Solving the linear system Ax=b for vector x):
    
  # Step 6:
  y[1] = b[1]
  
  # Step 7: 
  for (i in 2:n){
    y[i] = b[i] - sum(l2[i,1:i]*y[1:i])
  }
  
  # Step 8:
  for (i in 1:n){
    z[i] = (y[i]/d[i,i])
  }
  
  # Step 9:
  x[n] = z[n]
  
  # Step 10:
  for (i in (n-1):1){
    x[i] = z[i] - sum(l2[(i+1):n, i]*x[(i+1):n])
  }
  
  # Step 11:
  print("Matrix A (input) is:")
  print(A)
  print("Matrix L (lower triangular) is:")
  print(l2)
  print("Matrix D (diagonal) is:")
  print(d)
  print("Vector x (solution) is:")
  print(x) 
}

In [3]:
# Inputs:
A <- matrix(c(1, -3, 2, -2, -3, -3, 13, -8, 6, 7, 2, -8, 14, -7, 4, -2, 6, -7, 6, 2, -3, 7, 4, 2, 24), nrow=5, ncol=5)
b <- matrix(c(25, -81, -10, -25, -155), nrow=5, ncol=1)

In [4]:
# Should give: (x1 = 4, x2 = -5, x3 = -3, x4 = 0, x5 = -4)
ldlt_func(A,b)  

[1] "Matrix A (input) is:"
     [,1] [,2] [,3] [,4] [,5]
[1,]    1   -3    2   -2   -3
[2,]   -3   13   -8    6    7
[3,]    2   -8   14   -7    4
[4,]   -2    6   -7    6    2
[5,]   -3    7    4    2   24
[1] "Matrix L (lower triangular) is:"
     [,1] [,2]       [,3] [,4] [,5]
[1,]    1  0.0  0.0000000    0    0
[2,]   -3  1.0  0.0000000    0    0
[3,]    2 -0.5  1.0000000    0    0
[4,]   -2  0.0 -0.3333333    1    0
[5,]   -3 -0.5  1.0000000   -1    1
[1] "Matrix D (diagonal) is:"
     [,1] [,2] [,3] [,4] [,5]
[1,]    1    0    0    0    0
[2,]    0    4    0    0    0
[3,]    0    0    9    0    0
[4,]    0    0    0    1    0
[5,]    0    0    0    0    4
[1] "Vector x (solution) is:"
     [,1]
[1,]    4
[2,]   -5
[3,]   -3
[4,]    0
[5,]   -4
